<a href="https://colab.research.google.com/github/jcandane/CI_Theory/blob/main/FullCI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Full Configuration Interaction (in PySCF)

In [ ]:
!pip install pyscf 

     |████████████████████████████████| 37.5 MB 1.2 MB/s 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyscf_interface import *
from itertools import combinations, permutations
import pyscf
from pyscf import fci

π = np.pi
α = 0.007297352
c = 1.0/α
np.set_printoptions(precision=4, linewidth=200, threshold=2000, suppress=True)

In [ ]:
def get_Πso(uhf, entire=False):
    
    HA = np.einsum("AB, Ap, Bq -> pq", uhf.H, uhf.Ca, uhf.Ca, optimize=True)
    HB = np.einsum("AB, Ap, Bq -> pq", uhf.H, uhf.Cb, uhf.Cb, optimize=True)
    
    ΠAA  = np.einsum("ABCD, Ap, Bq, Cr, Ds -> pqrs", uhf.I, uhf.Ca, uhf.Ca, uhf.Ca, uhf.Ca, optimize=True) ## J
    ΠAA -= np.einsum("ADCB, Ap, Bq, Cr, Ds -> pqrs", uhf.I, uhf.Ca, uhf.Ca, uhf.Ca, uhf.Ca, optimize=True) ## K
    ΠBB  = np.einsum("ABCD, Ap, Bq, Cr, Ds -> pqrs", uhf.I, uhf.Cb, uhf.Cb, uhf.Cb, uhf.Cb, optimize=True) ## J
    ΠBB -= np.einsum("ADCB, Ap, Bq, Cr, Ds -> pqrs", uhf.I, uhf.Cb, uhf.Cb, uhf.Cb, uhf.Cb, optimize=True) ## K
    ΠAB  = np.einsum("ABCD, Ap, Bq, Cr, Ds -> pqrs", uhf.I, uhf.Ca, uhf.Ca, uhf.Cb, uhf.Cb, optimize=True)
    ΠBA  = np.einsum("ABCD, Ap, Bq, Cr, Ds -> pqrs", uhf.I, uhf.Cb, uhf.Cb, uhf.Ca, uhf.Ca, optimize=True)
    
    if entire:
        H = np.stack((HA, HB))
        Π = np.stack(( np.stack((ΠAA, ΠAB)), np.stack((ΠBA, ΠBB)) ))
        return Π, H
    
    else:
        return ΠAA, ΠAB, ΠBA, ΠBB, HA, HB

def givenΛgetB(ΛA, ΛB, N_mo):
  "Given Λ (i occupied orbitals for each determinant) get B (binary rep.)"

  Binary  = np.zeros((ΛA.shape[0], 2, N_mo), dtype=np.int8)
  for I in range(len(Binary)):
      Binary[I, 0, ΛA[I,:]] = 1
      Binary[I, 1, ΛB[I,:]] = 1

  return Binary

def givenBgetΛ(B):
  "Given B (entire MO binary occupation) get Λ (i occupied)"

  numA = len( (B[0,0])[ B[0,0] == 1 ] ) ## count num. of A occupied
  numB = len( (B[0,1])[ B[0,1] == 1 ] ) ## count num. of B occupied

  ΛA = np.zeros((B.shape[0], numA), dtype=np.int8)
  ΛB = np.zeros((B.shape[0], numB), dtype=np.int8)
  for I in range(len(Λ)):
    ΛA[I] = np.where(B == 1)[1]
    ΛB[I] = np.where(B == 1)[1]

  return ΛA, ΛB

def Ext(A, N, numtype=np.int8):
    if A.ndim == 1:
      return np.einsum("I, j -> Ij", A, np.ones( N, dtype=numtype))
    if A.ndim == 2:
      return np.einsum("uI, j -> uIj", A, np.ones( N, dtype=numtype))

def SpinOuterProduct(A, B):
  ΛA = np.einsum("Ii, J -> IJi", A, np.ones(B.shape[0], dtype=np.int8)).reshape( (A.shape[0]*B.shape[0], A.shape[1]) )
  ΛB = np.einsum("Ii, J -> JIi", B, np.ones(A.shape[0], dtype=np.int8)).reshape( (A.shape[0]*B.shape[0], A.shape[1]) )
  return ΛA, ΛB

#### Now here lets define our Molecule 

In [ ]:
#Z_HH = np.array(["He", "H", "H"])
#R_HH = np.array([[ 0., 0., 0.], [ 0., 1.5, 0.], [0., 0., 1.5]])
#Z_HH = np.array(["H", "H"]) #np.array(["Be", "H", "H"])
#R_HH = np.array([[ 0., 1.5, 0.], [0., 0., 1.5]])  #np.array([[ 0., 0., 0.], [ 0., 1.5, 0.], [0., 0., 1.5]])
Z_HH = np.array(["H", "H", "H", "H"])
R_HH = np.array([[ 0., 0., 0.], [ 1.5, 0.0, 0.], [ 0., 1.5, 0.], [0., 0., 1.5]])
#Z_HH = np.array(["He", "He", "He", "O"])
#R_HH = np.array([[ 0., 0., 0.], [ 1.5, 0.0, 0.], [ 0., 1.5, 0.], [0., 0., 1.5]])
#Z_HH = np.array(["He", "H", "He", "H"])
#R_HH = np.array([[ 0., 0., 0.], [ 1.5, 0.0, 0.], [ 0., 1.5, 0.], [0., 0., 1.5]])

#Z_HH = np.array(["H", "H", "H", "H"])
#R_HH = np.array([[ 0., 0., 0.], [ 1.5, 0.0, 0.], [ 0., 1.5, 0.], [0., 0., 1.5]])

#Z_HH = np.array(["H", "H"])
#R_HH = np.array([[ 0., 0., 0.], [ 0., 0.0, 2.3]])

HH = pyscf_UHF()
HH.initialize(xyz=R_HH, Z=Z_HH, charge=0, basis="sto-3g") ##-1
HH.Calc()

#print(HH.__dict__)

cisolver = pyscf.fci.FCI(HH.pyscfuhf,singlet=False)
cisolver.nroots = 100 # 100
cisolver.spin = 0
cisolver.davidson_only = False
#cisolver.singlet=False
pyscf_fci_energy = cisolver.kernel()[0]

In [ ]:
#print(cisolver.__dict__)
nuclear_rep = HH.nuclei_energy
evals = cisolver.eci
#print(evals)
#print(len(evals))
evecs = np.asarray(cisolver.ci).reshape(len(evals),len(evals))
#print(evecs)
HFCI = evecs.T@np.diag(evals-0.*nuclear_rep)@evecs
#print(HFCI)
HFCI = evecs.T@np.diag(evals-nuclear_rep)@evecs
#print(HFCI)
w, c = np.linalg.eigh(HFCI)
print(nuclear_rep)

1.8067240104218323


### Slater-Condon Preperation

CI Combinatorics

In [ ]:
Π, H = get_Πso(HH, entire=True)

N  = len(HH.S) ## number of MOs
Na = HH.Na
Nb = HH.Nb

O_α = np.asarray( list(combinations(  np.arange(0, N, 1, dtype=np.int8)  , Na ) ) ) ### all possible combinations of Nα electrons in N MO-orbitals!
O_β = np.asarray( list(combinations(  np.arange(0, N, 1, dtype=np.int8)  , Nb ) ) )  

In [ ]:
refex = np.array([0, 1, 0, 1, 1, 0, 0], dtype=np.int8)
refMR = np.array([[0, 1, 0, 1, 1, 0, 0],[0, 1, 1, 1, 0, 0, 0],[0, 1, 0, 0, 1, 0, 1]], dtype=np.int8)

def CI_Choice(reference, m):

  ### if reference is 1D
  if reference.ndim == 1: 
    Λ = np.where(reference == 1)[0]
    V = np.where(reference == 0)[0]

    if len(Λ) > m-1:
      Λ_sample = np.asarray( list(combinations(  Λ  , len(Λ) - m ) ) , dtype=np.int8)
      V_sample = np.asarray( list(combinations(  V  , m          ) ) , dtype=np.int8)

      a_Ii = np.empty((len(Λ_sample)*len(V_sample), len(Λ) ), dtype=np.int8)
      a_Ii[:,:len(Λ) - m ] = np.repeat(Λ_sample, len(V_sample), axis=0)
      a_Ii[:, len(Λ) - m:] = np.repeat(V_sample, len(Λ_sample), axis=1).reshape((len(Λ_sample)*len(V_sample), V_sample.shape[1]), order="F")
      a_Ii = np.sort(a_Ii, axis=1)
      return np.unique(a_Ii, axis=0)
    
    else:
      return None

  ### if reference is 2D (this is MR-CI)
  if reference.ndim == 2:
    if len(np.where(reference[0] == 1)[0]) > m-1:
      for i in range(len(reference)):
        Λ = np.where(reference[i] == 1)[0]
        V = np.where(reference[i] == 0)[0]

        Λ_sample = np.asarray( list(combinations(  Λ  , len(Λ) - m ) ) , dtype=np.int8)
        V_sample = np.asarray( list(combinations(  V  , m          ) ) , dtype=np.int8)

        a_new = np.empty((len(Λ_sample)*len(V_sample), len(Λ) ), dtype=np.int8)
        a_new[:,:len(Λ) - m ] = np.repeat(Λ_sample, len(V_sample), axis=0)
        a_new[:, len(Λ) - m:] = np.repeat(V_sample, len(Λ_sample), axis=1).reshape((len(Λ_sample)*len(V_sample), V_sample.shape[1]), order="F")
        a_new = np.sort(a_new, axis=1) 

        if i == 0:
          a_Ii = a_new
        else:
          a_Ii = np.append(a_Ii, a_new, axis=0)
      
      return np.unique(a_Ii, axis=0)

    else:
      return None

print( CI_Choice(refex, 3) )
print( CI_Choice(refMR, 3) )

[[0 2 5]
 [0 2 6]
 [0 5 6]
 [2 5 6]]
[[0 2 3]
 [0 2 5]
 [0 2 6]
 [0 3 5]
 [0 4 5]
 [0 4 6]
 [0 5 6]
 [2 3 5]
 [2 5 6]
 [4 5 6]]


Differences

In [ ]:
ΛA, ΛB = SpinOuterProduct(O_α, O_β)

Binary         = givenΛgetB(ΛA, ΛB, N)
Difference     = np.einsum("Isp, J -> IJsp", Binary, np.ones(len(Binary), dtype=np.int8)) - np.einsum("Isp, J -> JIsp", Binary, np.ones(len(Binary), dtype=np.int8))
Sum            = np.einsum("Isp, J -> IJsp", Binary, np.ones(len(Binary), dtype=np.int8)) + np.einsum("Isp, J -> JIsp", Binary, np.ones(len(Binary), dtype=np.int8))
SpinDifference = np.einsum("IJsp -> IJs", np.abs(Difference))//2

### Get the Sign for each Configuration
signA = np.cumsum( Binary[:,0,:] , axis=1)
signB = np.cumsum( Binary[:,1,:] , axis=1)
for I in range(len(ΛA)):
  signA[I, ΛA[I]] = np.arange(0, Na, 1)
  signB[I, ΛB[I]] = np.arange(0, Nb, 1)

Γ_IAp = ( (-1)**(signA) ).astype(np.int8)
Γ_IBp = ( (-1)**(signB) ).astype(np.int8)

In [ ]:
ΛA.shape

(36, 2)

### Slater-Condon Rules 0 \& 3

In [ ]:
AB = Ext(ΛA, Nb)
BA = Ext(ΛB, Na)
AA = Ext(ΛA, Na)
BB = Ext(ΛB, Nb)

Rule1  = np.einsum("Ii  -> I", H[0, ΛA, ΛA]) 
Rule1 += np.einsum("Ii  -> I", H[1, ΛB, ΛB])
Rule1 += np.einsum("Iij -> I", Π[0, 0, AA, AA, AA.swapaxes(1,2), AA.swapaxes(1,2)])/2
Rule1 += np.einsum("Iij -> I", Π[1, 0, BA, BA, AB.swapaxes(1,2), AB.swapaxes(1,2)])/2
Rule1 += np.einsum("Iij -> I", Π[0, 1, AB, AB, BA.swapaxes(1,2), BA.swapaxes(1,2)])/2
Rule1 += np.einsum("Iij -> I", Π[1, 1, BB, BB, BB.swapaxes(1,2), BB.swapaxes(1,2)])/2

H_CI = np.diag(Rule1)

### Slater-Condon Rule 1

In [ ]:
### indices for 1-difference
I_A, J_A = np.where( np.all(SpinDifference==np.array([1,0], dtype=np.int8), axis=2) )
I_B, J_B = np.where( np.all(SpinDifference==np.array([0,1], dtype=np.int8), axis=2) )

A_t  = (np.where( Difference[I_A, J_A, 0] ==  1 )[1]).astype(np.int8)
A    = (np.where( Difference[I_A, J_A, 0] == -1 )[1]).astype(np.int8)
B_t  = (np.where( Difference[I_B, J_B, 1] ==  1 )[1]).astype(np.int8)
B    = (np.where( Difference[I_B, J_B, 1] == -1 )[1]).astype(np.int8)
CA_i = (np.where( Sum[I_A, J_A, 0] == 2 )[1]).reshape(len(I_A), Na-1)
CB_i = (np.where( Sum[I_B, J_B, 1] == 2 )[1]).reshape(len(I_B), Nb-1)

if I_A.size > 0:
  H_CI[I_A, J_A]  = H[0, A, A_t]
  H_CI[I_A, J_A] += np.einsum("In -> I", Π[0, 0, Ext(A, Na-1), Ext(A_t, Na-1), CA_i, CA_i])
  H_CI[I_A, J_A] += np.einsum("Ii -> I", Π[0, 1, Ext(A, Nb  ), Ext(A_t, Nb  ), ΛB[I_A], ΛB[J_A]])
  H_CI[I_A, J_A] *= Γ_IAp[I_A, A_t] * Γ_IAp[J_A, A]

if I_B.size > 0:
  H_CI[I_B, J_B]  = H[1, B, B_t]
  H_CI[I_B, J_B] += np.einsum("In -> I", Π[1, 1, Ext(B, Nb-1), Ext(B_t, Nb-1), CB_i, CB_i])
  H_CI[I_B, J_B] += np.einsum("Ii -> I", Π[1, 0, Ext(B, Na  ), Ext(B_t, Na  ), ΛA[I_B], ΛA[J_B]])
  H_CI[I_B, J_B] *= Γ_IBp[I_B, B_t] * Γ_IBp[J_B, B]

### Slater-Condon Rule 2

In [ ]:
### get indices with differences by 2
I_AA, J_AA = np.where( np.all(SpinDifference==np.array([2,0], dtype=np.int8), axis=2) )
I_BB, J_BB = np.where( np.all(SpinDifference==np.array([0,2], dtype=np.int8), axis=2) )
I_AB, J_AB = np.where( np.all(SpinDifference==np.array([1,1], dtype=np.int8), axis=2) )

AA   = np.where( Difference[I_AA, J_AA, 0] == -1)[1].reshape(len(I_AA),2).T
AA_t = np.where( Difference[I_AA, J_AA, 0] ==  1)[1].reshape(len(I_AA),2).T
BB   = np.where( Difference[I_BB, J_BB, 1] == -1)[1].reshape(len(I_BB),2).T
BB_t = np.where( Difference[I_BB, J_BB, 1] ==  1)[1].reshape(len(I_BB),2).T
AB   = np.asarray([ np.where( Difference[I_AB, J_AB, 0] == -1 )[1], np.where( Difference[I_AB, J_AB, 1] == -1 )[1] ])
AB_t = np.asarray([ np.where( Difference[I_AB, J_AB, 0] ==  1 )[1], np.where( Difference[I_AB, J_AB, 1] ==  1 )[1] ])

if I_AA.size > 0: 
    H_CI[I_AA, J_AA]  = Π[0, 0, AA[0], AA_t[0], AA[1], AA_t[1]]
    H_CI[I_AA, J_AA] *= Γ_IAp[I_AA, AA[0]] * Γ_IAp[I_AA, AA_t[0]] * Γ_IAp[J_AA, AA[0]] * Γ_IAp[J_AA, AA_t[0]]
if I_BB.size > 0: 
    H_CI[I_BB, J_BB]  = Π[1, 1, BB[0], BB_t[0], BB[1], BB_t[1]]
    H_CI[I_BB, J_BB] *= Γ_IBp[I_BB, BB[1]] * Γ_IBp[J_BB, BB_t[1]] * Γ_IBp[I_BB, BB[0]] * Γ_IBp[J_BB, BB_t[0]]
if I_AB.size > 0:
    H_CI[I_AB, J_AB]  = Π[0, 1, AB[0], AB_t[0], AB[1], AB_t[1]]
    H_CI[I_AB, J_AB] *= Γ_IBp[I_AB, AB[1]] * Γ_IBp[J_AB, AB_t[1]] * Γ_IAp[I_AB, AB[0]] * Γ_IAp[J_AB, AB_t[0]]

# Compare

In [ ]:
nuclear_rep = HH.nuclei_energy
e_fci_my, mywavefunctions = np.linalg.eigh(H_CI)

print("pyscf : " + str(pyscf_fci_energy[0] ))
print("Mine  : " + str( e_fci_my[0] + nuclear_rep ))

pyscf : -1.9290094636231134
Mine  : -1.9290094636231099


In [ ]:
H_CI.shape

(36, 36)